# 实现简单前馈神经网络
## 包引入

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [3]:
# 如果有GPU,使用GPU
device = torch.device('cuda' if  torch.cuda.is_available() else 'cpu')

## 引入数据集并创建数据读取器

In [7]:
# 指定超参数
batch_size = 100


train_dataset = torchvision.datasets.MNIST(root='../../../datasets',
                                          train=True,
                                          transform=transforms.ToTensor(),
                                          download=True)
test_dataset = torchvision.datasets.MNIST(root='../../../datasets',
                                          train=False,
                                          transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

## 创建神经网络

In [8]:
# 指定超参数
input_size  = 28 * 28
hidden_size = 128
num_classes = 10
learning_rate = 0.001


class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1  = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2  = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## 训练模型

In [9]:
# 定义超参数
num_epochs = 5

total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # 将参数移动到运行设备上（CPU or GPU）
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 梯度求导并优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # 每一定轮数打印模型训练结果
        if (i+1) % 100 == 0:
            print('epoch [{}/{}], step [{}/{}], Loss:{:.4f}'
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

epoch [1/5], step [100/600], Loss:0.5027
epoch [1/5], step [200/600], Loss:0.3462
epoch [1/5], step [300/600], Loss:0.2358
epoch [1/5], step [400/600], Loss:0.3103
epoch [1/5], step [500/600], Loss:0.2228
epoch [1/5], step [600/600], Loss:0.2406
epoch [2/5], step [100/600], Loss:0.2756
epoch [2/5], step [200/600], Loss:0.3204
epoch [2/5], step [300/600], Loss:0.1271
epoch [2/5], step [400/600], Loss:0.1647
epoch [2/5], step [500/600], Loss:0.0535
epoch [2/5], step [600/600], Loss:0.1747
epoch [3/5], step [100/600], Loss:0.2171
epoch [3/5], step [200/600], Loss:0.1762
epoch [3/5], step [300/600], Loss:0.1201
epoch [3/5], step [400/600], Loss:0.0420
epoch [3/5], step [500/600], Loss:0.1240
epoch [3/5], step [600/600], Loss:0.0585
epoch [4/5], step [100/600], Loss:0.1641
epoch [4/5], step [200/600], Loss:0.1371
epoch [4/5], step [300/600], Loss:0.0397
epoch [4/5], step [400/600], Loss:0.1395
epoch [4/5], step [500/600], Loss:0.0517
epoch [4/5], step [600/600], Loss:0.0489
epoch [5/5], ste

## 测试模型

In [11]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images       = images.reshape(-1, 28*28).to(device)
        labels       = labels.to(device)
        outputs      = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total        += labels.size(0) 
        correct      += (predicted == labels).sum().item()
        
    print('accuracy is : {}%'.format(100 * correct / total))

accuracy of is : 97.43%
